In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow

In [2]:
# Load the iris dataset
df = pd.read_csv("../data/cleaned_data.csv", low_memory = False)



In [3]:
#mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [4]:
#mlflow.set_experiment("check-localhost-connection")

#with mlflow.start_run():
 #   mlflow.log_metric("foo", 1)
 #   mlflow.log_metric("bar", 2)

In [5]:


# Splitting the DataFrame into X (features) and y (labels)
X = df.drop(columns=['loan_status'])  # Drop the 'label' column to get the features
y = df['loan_status']  # Get only the 'label' column as the target variable

# Printing the first few rows of X and y for verification
print("Features (X):")
print(X.head())
print("\nLabels (y):")
print(y.head())


Features (X):
        id  member_id  loan_amnt  funded_amnt  term  int_rate  installment  \
0  1077501    1296599     5000.0       5000.0    36     10.65       162.87   
1  1077430    1314167     2500.0       2500.0    60     15.27        59.83   
2  1077175    1313524     2400.0       2400.0    36     15.96        84.33   
3  1076863    1277178    10000.0      10000.0    36     13.49       339.31   
4  1075358    1311748     3000.0       3000.0    60     12.69        67.79   

   grade  sub_grade  emp_length  ...  total_acc  out_prncp   total_pymnt  \
0      2          7          11  ...        9.0        0.0   5861.071414   
1      3         14           0  ...        4.0        0.0   1008.710000   
2      3         15          11  ...       10.0        0.0   3003.653644   
3      3         11          11  ...       37.0        0.0  12226.302212   
4      2         10           1  ...       38.0      766.9   3242.170000   

   total_rec_prncp  total_rec_late_fee  acc_now_delinq  loan

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
eval_set=[(X_test, y_test)]
clf2 = xgb.sklearn.XGBClassifier(
    objective="binary:logistic", 
    learning_rate=0.05, 
    seed=9616, 
    max_depth=20, 
    gamma=10, 
    n_estimators=500)

clf2.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=eval_set, verbose=True)

C:\Users\furka\PycharmProjects\Credit_Risk\venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\furka\PycharmProjects\Credit_Risk\venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBoostError: [22:08:38] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\../data/gradient_index.h:94: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`

In [ ]:
probabilities = clf2.predict_proba(X_test)
positive_probabilities = probabilities[:, 1]
positive_probabilities

In [ ]:
y_pred = clf2.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve and ROC AUC
fpr, tpr, thresholds = roc_curve(y_test, positive_probabilities)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
